### redis-api
- Redis
    - using `docker run -p 6379:6379 -it --rm redislabs/redismod`
    - `redismod` includes the graph and RedisJSON modules
- Docs
    - RedisJSON: https://redis.io/docs/stack/json/
    - https://redis.io/commands/graph.query/
    - https://docs.redis.com/latest/modules/redisgraph/redisgraph-quickstart/
    - https://redis.js.org/documentation/json/
- Redis cli
    - INFO
    
```
redis-cli -h 192.168.1.166 -p 6379
> INFO

# Server
redis_version:6.2.6

# Modules
module:name=ai,ver=10205,api=1,filters=0,usedby=[],using=[],options=[handle-io-errors]
module:name=ReJSON,ver=20011,api=1,filters=0,usedby=[search|graph],using=[],options=[handle-io-errors]
module:name=graph,ver=20815,api=1,filters=0,usedby=[],using=[ReJSON],options=[]
module:name=bf,ver=20209,api=1,filters=0,usedby=[],using=[],options=[]
module:name=rg,ver=10204,api=1,filters=1,usedby=[rg],using=[rg],options=[]
module:name=search,ver=999999,api=1,filters=0,usedby=[],using=[ReJSON],options=[handle-io-errors]
module:name=timeseries,ver=10616,api=1,filters=0,usedby=[],using=[],options=[handle-io-errors]
```

In [1]:
dotenv = require('dotenv')

{ config: [Function: config], parse: [Function: parse] }

In [2]:
process.version

'v16.19.0'

In [3]:
dotenv.config()
console.log(process.env.REDIS_HOST)

redis://192.168.1.132:6379


In [4]:
redisClient = require('./redis/redis-client.js')

connecting to: undefined


{
  _events: [Object: null prototype] {},
  _eventsCount: 0,
  _maxListeners: undefined,
  commandOptions: [Function: commandOptions],
  select: [AsyncFunction: SELECT],
  subscribe: [Function: SUBSCRIBE],
  pSubscribe: [Function: PSUBSCRIBE],
  unsubscribe: [Function: UNSUBSCRIBE],
  pUnsubscribe: [Function: PUNSUBSCRIBE],
  quit: [Function: QUIT],
  multi: [Function: MULTI],
  bf: {},
  cms: {},
  cf: {},
  tDigest: {},
  topK: {},
  graph: {},
  json: {},
  ft: {},
  ts: {},
  getKeysAsync: [AsyncFunction: getKeysAsync],
  getAsync: [AsyncFunction: getAsync],
  setAsync: [AsyncFunction: setAsync],
  deleteAsync: [AsyncFunction: deleteAsync],
  queryGraphAsync: [AsyncFunction: queryGraphAsync],
  getJsonAsync: [AsyncFunction: getJsonAsync],
  setJsonAsync: [AsyncFunction: setJsonAsync],
  deleteJsonAsync: [AsyncFunction: deleteJsonAsync],
  [Symbol(kCapture)]: false
}

In [5]:
(async () => {
    result = await redisClient.getKeysAsync()
    console.log(result)
})()

Promise { <pending> }

[
  'lima:metadata:gpt3text/robo-chitchat-jan-2023',
  'lima:trx:42d601bf-a985-478f-afc0-4ffa196cc9ce',
  'lima:trx:b0cd3952-344c-42f9-9c18-bd217245c3fc',
  'lima:metadata:luis/robo-dispatch',
  'lima:anx:bf8c7e8d-c88a-437c-93e8-b105cedb410c'
]


In [ ]:
(async () => {
    result = await redisClient.queryGraphAsync(`MATCH (n) return n`)
    console.log(result)
})()

In [ ]:
(async () => {
    json = { "employee": { "name": "alpha", "age": 24,"single": true }  }
    result = await redisClient.setJsonAsync('employee_profile', json)
    console.log(result)
})()

In [ ]:
(async () => {
    const options = {
        path: [
            '.employee.name',
            '.employee.age'
        ]
    }
    result = await redisClient.getJsonAsync('employee_profile', options)
    console.log(result)
})()

In [ ]:
(async () => {
    const options = {}
    result = await redisClient.getJsonAsync('lima:trx:416df429-dc22-4bd0-af35-c70be9d0d97a', options)
    console.log(result)
})()

In [ ]:
(async () => {
    result = await redisClient.setAsync('mykey', 'Hello, world.')
    console.log(result)
})()

In [ ]:
(async () => {
    result = await redisClient.getAsync('mykey')
    console.log(result)
})()

In [ ]:
trx1 = {
    clientId: 'client-id',
    sessionId: 'session-id',
    input: 'do you like ice cream->',
    inputData: 'input-data',
    type: 'user',
    serviceType: 'gpt3text',
    appName: 'gpt3text/robo-chitchat-jan-2023',
    accountId: 'robot1',
    environment: 'environment',
}

trx2 = {
    clientId: 'client-id',
    sessionId: 'session-id',
    input: 'what time is it->',
    inputData: 'input-data',
    type: 'user',
    serviceType: 'gpt3text',
    appName: 'gpt3text/robo-chitchat-jan-2023',
    accountId: 'robot2',
    environment: 'environment',
}

In [ ]:
(async () => {
    result = await redisClient.setJsonAsync('trx:1', trx1)
    console.log(result)
})()

In [ ]:
(async () => {
    result = await redisClient.setJsonAsync('trx:2', trx2)
    console.log(result)
})()

In [ ]:
(async () => {
    result = await redisClient.getJsonAsync('lima:trx:4cbdeb11-717e-4fbc-b4aa-dc310a6dc532')
    console.log(result)
})()

In [ ]:
// (async () => {
//     result = await redisClient.deleteJsonAsync('trx:2')
//     console.log(result)
// })()

### Search

In [ ]:
redisClient.ft.create('lima:idx:trx', {
        "$.id": {
            "type": "TEXT",
            "AS": "id"
        },
        "$.sessionId": {
            "type": "TEXT",
            "AS": "sessionId"
        },
        "$.serviceType": {
            "type": "TEXT",
            "AS": "serviceType"
        },
        "$.accountId": {
            "type": "TEXT",
            "AS": "accountId"
        }
    }, {
      ON: 'JSON',
      PREFIX: 'lima:trx:'
    })
    .then(result => console.log(result))
    .catch(error => console.log(error))

In [ ]:
redisClient.ft.search('lima:idx:trx', '*')
    .then(result => console.log(util.inspect(result, {showHidden: false, depth: 5, colors: true})))
    .catch(error => console.log(error))

In [ ]:
redisClient.ft.search('lima:idx:trx', `@serviceType:'luis'`)
    .then(result => console.log(result))
    .catch(error => console.log(error))

In [ ]:
redisClient.ft.search('idx:trx', `@accountId:'robot2'`)
    .then(result => console.log(result))
    .catch(error => console.log(error))

In [ ]:
redisClient.ft.search('idx:trx', `@accountId:'robot2' @serviceType:'luis'`)
    .then(result => console.log(result))
    .catch(error => console.log(error))

In [ ]:
redisClient.ft.search('idx:trx', `@accountId:'robot2' @serviceType:'(luis|gpt3text)'`)
    .then(result => console.log(result))
    .catch(error => console.log(error))

In [ ]:
redisClient.ft.search('idx:trx', `@accountId:'(robot1|robot2)' @serviceType:'(luis|gpt3text)'`)
    .then(result => console.log(result))
    .catch(error => console.log(error))

In [ ]:
redisClient.ft.search('lima:idx:anx', `*`)
    .then(result => console.log(result))
    .catch(error => console.log(error))

In [ ]:
redisClient.ft._list()
    .then(result => console.log(result))
    .catch(error => console.log(error))

In [ ]:
redisClient.ft.info('lima:idx:metadata')
    .then(result => console.log(result))
    .catch(error => console.log(error))

In [ ]:
redisClient.ft.dropIndex('lima:idx:anx')
    .then(result => console.log(result))
    .catch(error => console.log(error))